In [ ]:
# Standard library imports
import sys

# Third party imports
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
import seaborn as sns
import numpy as np
import prince
from networkx.algorithms import bipartite
from matplotlib.patches import Patch
import community as community_louvain
from netgraph import Graph

# Local application imports
sys.path.insert(0, '../Utility files')
from corg import DiscoverDimension, BenchmarkDimension
import utils2
from utils2 import *
import ca_pipeline
from ca_pipeline import PipelineCorAnalysis
from matplotlib.ticker import FuncFormatter


### 1. Create a labeled subset to use in the CORG approach

- In this notebook, I am creating a labelled version of the full edgelist. Markers are labeled as high or low SES based on expert knowledge. This is subsequently to be used as input in the CORG method for model validation.

This part of the repository was performed in order to do a marker selection Process that has not been written about in the thesis because it did not work. 
The aim was to label certain marers as high or low SES based on assumptions, and identify the dimension that best separated the classififed markers. This is included here, because the labels needed to exist in the coordinate files. 

In [ ]:
# Create empty data frame to use for labeling


# # load data
# path = '/home/livtollanes/NewData/DataWrangling/french_edgelist_marker_threshold.csv'

# req_cols = ['marker_id', 'follower_id', 'twitter_name','followers','french_followers', 'type', 'type2']
# dtypes = {'marker_id': 'object',
#           'follower_id': 'object',
#           'twitter_name': 'object',
#           'followers': 'int64',
#           'french_followers': 'int64',
#           'type': 'object',
#           'type2': 'object'}


# edgelist_CORG = pd.read_csv(path, usecols=req_cols, dtype=dtypes)

# #Separate my data frame into dictionary of types
# def create_dict(df, key_col, value_col):
#     unique_df = df.drop_duplicates(subset=[key_col, value_col])
#     return unique_df.groupby(key_col)[value_col].apply(list).to_dict()


# df_dict = create_dict(edgelist_CORG, 'type', 'twitter_name')
# df_dict2 = create_dict(edgelist_CORG, 'type2', 'twitter_name')


# for key, value in df_dict.items():
#     value = list(set(value))
#     print(f"Key: {key}")
#     print(f"Value: {value}\n")



# # Create a csv file to use for labeling 
# rows = []
# for key, values in df_dict.items():
#     for value in values:
#         row = {'type': key, 'twitter_name': value, 'high': '', 'low': ''}
#         rows.append(row)

# # Now we write the rows to a CSV file
# keys = ['type', 'twitter_name', 'high', 'low']

# with open('/home/livtollanes/10.jan-thesis/labeldata/labeled_markers_empty.csv', 'w', newline='') as output_file:
#     dict_writer = csv.DictWriter(output_file, keys)
#     dict_writer.writeheader()
#     dict_writer.writerows(rows)

Now the empty df above was fille din externally in a google docs. 

In [ ]:
#Load filled data
# load data (this is the full list of markers together with the labels) 223 markers
path = '/home/livtollanes/10.jan-thesis/labeldata/filled_labels_markers.csv'

req_cols = ['type', 'twitter_name', 'high', 'low', 'other']
dtypes = {'type': 'object',
          'twitter_name': 'object',
          'high': 'object',
          'low': 'object',
          'other': 'object'}

labels_hl = pd.read_csv(path, usecols=req_cols, dtype=dtypes)

In [ ]:
#refomat the data
# I put 0: low, 1: high, NA: other
# Create the labeled marker df to input to the CORG method

# Create a copy of labels_hl
labels_CORG = labels_hl.copy()

# Create 'SES' column
labels_CORG['label'] = np.where(labels_CORG['high'] == '1', '1', 
                              np.where(labels_CORG['low'] == '1', '0', 'NA'))


#0: low, 1: high, NA: other

# Drop the rows where 'SES' is 'drop'
labels_CORG = labels_CORG[labels_CORG['label'] != 'drop']

# Keep only 'twitter_name' and 'SES' columns
labels_CORG = labels_CORG[['twitter_name', 'type', 'label']]

labels_CORG['label'].value_counts() # 22 H, 14 L. Looks correct

In [ ]:
# load full marker edgelist data. All follower relationships
path = '/home/livtollanes/NewData/DataWrangling/french_edgelist_marker_threshold.csv'

req_cols = ['marker_id', 'follower_id', 'twitter_name','followers','french_followers', 'type', 'type2']
dtypes = {'marker_id': 'object',
          'follower_id': 'object',
          'twitter_name': 'object',
          'followers': 'int64',
          'french_followers': 'int64',
          'type': 'object',
          'type2': 'object'}


all = pd.read_csv(path, usecols=req_cols, dtype=dtypes)



In [ ]:
labels_CORG_subset = labels_CORG[['twitter_name', 'label']]
all_labeled = all.merge(labels_CORG_subset, how='left', on='twitter_name')

#Save the labeled edgelist to csv in /home/livtollanes/NewData
#all_labeled.to_csv('/home/livtollanes/NewData/DataWrangling/labeled_edgelist_hl.csv', index=False)